In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [3]:
!pip install pycountry
!pip install pycountry-convert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 5.8 MB/s eta 0:00:00


In [4]:
import os
import pycountry

import pandas as pd
import scipy.stats as stats
import plotly.express as px
import pycountry_convert as pc


from roboflow import Roboflow
from sklearn.preprocessing import LabelEncoder

from drive.MyDrive.roboflow_configs.nk37_config import (
    API_KEY,
    WORKSPACE,
    DISEASED_TOMATOES_PROJECT_ID,
    DISEASED_TOMATOES_FILTERED_VERSION_NUMBER
  )


DRIVE_DIR = "/content/drive/MyDrive"
TABULAR_DATA_FILE_PATH = f"{DRIVE_DIR}/roboflow_configs/new_columns_general_and_7_diseases_202412011734.csv"

In [5]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(DISEASED_TOMATOES_PROJECT_ID)
version = project.version(DISEASED_TOMATOES_FILTERED_VERSION_NUMBER)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to inaturalist-diseased-tomatoes-3 in folder:: 100%|██████████| 608/608 [00:00<00:00, 1286.04it/s]


In [6]:
tabular_data_df = pd.read_csv(TABULAR_DATA_FILE_PATH)
tabular_data_df.head()

,observation_id,time_observed_at_date,observed_on_date,observed_on_year,observed_on_month,observed_on_week,observed_on_day,observed_on_hour,location,place_guess,...,original_height,license_code,attribution,latitude,longitude,actual_city_name,actual_admin1,actual_admin2,actual_cc,roboflow_file_name
0,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,1367,cc-by-nc-sa,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_0_potato_blight_phytophthora_infesta...
1,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,1152,cc-by-nc-sa,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_1_potato_blight_phytophthora_infesta...
2,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,1367,cc-by-nc-sa,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_2_potato_blight_phytophthora_infesta...
3,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,1367,cc-by-nc-sa,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_3_potato_blight_phytophthora_infesta...
4,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,1367,cc-by-nc-sa,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_4_potato_blight_phytophthora_infesta...


In [41]:
tabular_data_df['identifications_most_disagree'].value_counts()

,count
identifications_most_disagree,
False,12818
True,7


In [42]:
tabular_data_df['identifications_most_agree'].value_counts()

,count
identifications_most_agree,
True,11650
False,1175


In [7]:
# Function to retrieve file names without extensions
def get_filenames_without_extension(directory_path):
    filenames = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            filenames.append(os.path.splitext(file)[0])  # Strip the extension
    return filenames

# Define the dataset directories
directories = [
    "/content/inaturalist-diseased-tomatoes-3/train",
    "/content/inaturalist-diseased-tomatoes-3/test",
    "/content/inaturalist-diseased-tomatoes-3/valid"
]

# Collect all filenames without extensions
all_filenames = []
for dir_path in directories:
    all_filenames.extend(get_filenames_without_extension(dir_path))

# Example: Print first 10 filenames
print(all_filenames[:10])

['34455412_296_potato_blight_phytophthora_infestans_2019_10_16_15_europe_brussels_perwez_wallonia_province_du_brabant_wallon_BE_jpg.rf.c1291af8e7530ab3ed4417be423005ba', '34546770_294_potato_blight_phytophthora_infestans_2019_10_18_15_europe_brussels_perwez_wallonia_province_du_brabant_wallon_BE_jpg.rf.c2063c0d0348d51843629e29a324d596', '139744615_157_potato_blight_phytophthora_infestans_2022_10_22_0_europe_brussels_celles_wallonia_province_du_hainaut_BE_jpg.rf.1a560e1268a122f3a78ceca1416285c9', '93149623_330_potato_blight_phytophthora_infestans_2021_8_31_8_america_chicago_lincolnwood_illinois_cook_county_US_jpg.rf.e5c23c32a8bf66ac3a3eaf55b4b407fd', '147208279_9_potato_blight_phytophthora_infestans_2023_1_20_9_america_los_angeles_la_jolla_california_san_diego_county_US_jpg.rf.3fcf7c34a88da94e24a6c4ff6c9d5395', '140671982_144_potato_blight_phytophthora_infestans_2022_10_30_0_europe_brussels_sombreffe_wallonia_province_de_namur_BE_jpg.rf.e9448bb146accd2ad4af05cd94f16305', '187266742_101_

Remove _jpg.rf.identifer

In [8]:
# Function to clean filenames (remove `.rf.identifier` and `_jpg` suffix)
def clean_filenames(filenames):
    cleaned = []
    for name in filenames:
        # Remove .rf.* and _jpg suffix
        if ".rf." in name:
            name = name.split(".rf.")[0]
        if name.endswith("_jpg"):
            name = name[:-4]  # Remove the last 4 characters (_jpg)
        cleaned.append(name)
    return cleaned

# Collect all filenames without extensions
all_filenames = []
for dir_path in directories:
    all_filenames.extend(get_filenames_without_extension(dir_path))

# Clean the filenames to remove unwanted parts
cleaned_filenames = clean_filenames(all_filenames)

# Example: Print first 10 cleaned filenames
print(cleaned_filenames[:10])

['34455412_296_potato_blight_phytophthora_infestans_2019_10_16_15_europe_brussels_perwez_wallonia_province_du_brabant_wallon_BE', '34546770_294_potato_blight_phytophthora_infestans_2019_10_18_15_europe_brussels_perwez_wallonia_province_du_brabant_wallon_BE', '139744615_157_potato_blight_phytophthora_infestans_2022_10_22_0_europe_brussels_celles_wallonia_province_du_hainaut_BE', '93149623_330_potato_blight_phytophthora_infestans_2021_8_31_8_america_chicago_lincolnwood_illinois_cook_county_US', '147208279_9_potato_blight_phytophthora_infestans_2023_1_20_9_america_los_angeles_la_jolla_california_san_diego_county_US', '140671982_144_potato_blight_phytophthora_infestans_2022_10_30_0_europe_brussels_sombreffe_wallonia_province_de_namur_BE', '187266742_101_potato_blight_phytophthora_infestans_2023_10_12_18_europe_madrid_sada_galicia_provincia_da_coruna_ES', '250165869_23_potato_blight_phytophthora_infestans_2024_10_10_0_europe_brussels_grez-doiceau_wallonia_province_du_brabant_wallon_BE', '11

In [9]:
diseased_tomatoes_filenames_set = set(cleaned_filenames)
len(diseased_tomatoes_filenames_set)

584

In [10]:
diseased_tomatoes_df = tabular_data_df[tabular_data_df['roboflow_file_name'].isin(diseased_tomatoes_filenames_set)]
diseased_tomatoes_df.shape

(564, 34)

In [11]:
# Extract first and second words separated by '_'
tabular_data_df['identifier'] = tabular_data_df['roboflow_file_name'].apply(lambda x: '_'.join(x.split('_')[:2]))

# Create a set of unique identifiers from cleaned_filenames
cleaned_identifiers_set = set(['_'.join(name.split('_')[:2]) for name in cleaned_filenames])

# Filter rows that match the identifiers
diseased_tomatoes_df = tabular_data_df[tabular_data_df['identifier'].isin(cleaned_identifiers_set)]
print("Matching records:", diseased_tomatoes_df.shape)

Matching records: (584, 35)


In [12]:
import pandas as pd
import os

# INATURALIST_TO_OWN_ANNOTATIONS = {
#     'Spider Mites': 'Spider Mites',
#     'Tobacco Spider Mite': 'Spider Mites',
#     'Twospotted Spider Mite': 'Spider Mites',
#     'American Serpentine Leaf Miner': 'Leaf Miner',
#     'Leaf-miner Flies': 'Leaf Miner',
#     'Liriomyza leaf-mining flies': 'Leaf Miner',
#     'Vegetable Leafminer': 'Leaf Miner',
#     'potato blight': 'Late Blight',
#     'Early Blight': 'Early Blight',
#     'Tomato Leaf Mold': 'Leaf Mold',
#     'Septoria Leaf Spot of Tomato': 'Septoria',
#     'Tomato yellow leaf curl virus': 'Yellow Leaf Curl Virus',
# }

# # Example disease to category mapping
# INATURALIST_TO_OWN_ANNOTATIONS = {
#     'spider_mites': 'Spider Mites',
#     'tobacco_spider_mite': 'Spider Mites',
#     'twospotted_spider_mite': 'Spider Mites',
#     'american_serpentine_leaf_miner': 'Leaf Miner',
#     'leaf_miner_flies': 'Leaf Miner',
#     'liriomyza_leaf_mining_flies': 'Leaf Miner',
#     'vegetable_leafminer': 'Leaf Miner',
#     'potato_blight': 'Late Blight',
#     'early_blight': 'Early Blight',
#     'tomato_leaf_mold': 'Leaf Mold',
#     'septoria_leaf': 'Septoria',  # Mapping adjusted for two-word disease name
#     'tomato_yellow_leaf_curl_virus': 'Yellow Leaf Curl Virus',
# }


# Example disease to category mapping
INATURALIST_TO_OWN_ANNOTATIONS = {
    'spider_mites': 'Spider Mites',
    'tobacco_spider': 'Spider Mites',
    'twospotted_spider': 'Spider Mites',
    'american_serpentine': 'Leaf Miner',
    'leaf_miner': 'Leaf Miner',
    'liriomyza_leaf': 'Leaf Miner',
    'vegetable_leafminer': 'Leaf Miner',
    'potato_blight': 'Late Blight',
    'early_blight': 'Early Blight',
    'tomato_leaf': 'Leaf Mold',
    'septoria_leaf': 'Septoria',  # Mapping adjusted for two-word disease name
    'tomato_yellow': 'Yellow Leaf Curl Virus',
}


# Function to extract and map the disease name from the filename
def map_disease_from_filename(filename):
    # Split the filename by underscores

    parts = filename.replace('-', '_').split('_') # replace because of cases such as liriomyza_leaf-minning => but we only need two words

    # Extract the first two words as the disease name
    disease_name = '_'.join(parts[2:4])  # Take the 3rd and 4th parts as disease name

    # Convert disease name to lowercase
    disease_name = disease_name.lower().replace(' ', '_')  # Ensure proper format

    # Map to the disease category
    return INATURALIST_TO_OWN_ANNOTATIONS.get(disease_name, 'Unknown')


# Assuming your DataFrame is already loaded and contains the 'roboflow_file_name' column
diseased_tomatoes_df['disease'] = diseased_tomatoes_df['roboflow_file_name'].apply(map_disease_from_filename)

# Display the result
diseased_tomatoes_df.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,observation_id,time_observed_at_date,observed_on_date,observed_on_year,observed_on_month,observed_on_week,observed_on_day,observed_on_hour,location,place_guess,...,attribution,latitude,longitude,actual_city_name,actual_admin1,actual_admin2,actual_cc,roboflow_file_name,identifier,disease
0,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_0_potato_blight_phytophthora_infesta...,177163857_0,Late Blight
1,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_1_potato_blight_phytophthora_infesta...,177163857_1,Late Blight
2,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_2_potato_blight_phytophthora_infesta...,177163857_2,Late Blight
3,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_3_potato_blight_phytophthora_infesta...,177163857_3,Late Blight
4,177163857,2023-08-06T13:46:00+10:00,2023-08-06,2023,8,31,6,13,"-30.3512715382,153.0816898073","Coffs Harbour NSW, Australia",...,"(c) Nick Lambert, some rights reserved (CC BY-...",-30.351272,153.08169,Toormina,New South Wales,Coffs Harbour,AU,177163857_4_potato_blight_phytophthora_infesta...,177163857_4,Late Blight


So above did not work due to next cases:
 - 232205847_65_potato_blight_phytophthora_infestans_2024_7_27_20_europe_paris_ecouche_lower_normandy_departement_de_l-orne_FR
 - 232205847_65_potato_blight_phytophthora_infestans_2024_7_27_20_europe_paris_ecouche_lower_normandy_departement_de_l'orne_FR


So because we might have ' which was replaced by roboflow or some other symbols
we decided to stick to integer identifiers

After mannually looking at values we may confidently say that

In [13]:
# Count occurrences of each country for all diseases
all_diseases_country_counts = diseased_tomatoes_df['actual_cc'].value_counts()

# Display top countries
print("Top countries for all diseases:")
print(all_diseases_country_counts)

Top countries for all diseases:
actual_cc
US    169
BE    105
TW     41
DE     36
CZ     34
RU     27
PL     26
BY     20
AT     13
IT     11
PT     11
CA     11
ES     10
FR      9
FI      8
GB      7
AU      7
UA      6
ZA      6
CO      5
CK      5
BR      4
IN      3
GM      2
CR      2
LT      2
NZ      2
CN      1
NL      1
Name: count, dtype: int64


In [14]:
# Create the scatter_geo plot
fig = px.scatter_geo(diseased_tomatoes_df,
                     lon='longitude',
                     lat='latitude',
                     color='disease',  # Optionally color by disease or another column
                     hover_name='disease',  # Information to show when hovering over a point
                     hover_data={'actual_cc': True},  # Add 'actual_cc' to hover data
                     title="Global Disease Observations",
                     projection="orthographic",  # Use 'orthographic' for a globe-like appearance
                     color_continuous_scale=px.colors.sequential.Plasma)

# Customize layout for better display and to show country borders and names
fig.update_layout(
    geo=dict(
        showland=True,  # Show land on the map
        landcolor="lightgray",  # Land color
        showocean=True,  # Show ocean
        oceancolor="lightblue",  # Ocean color
        showcoastlines=True,  # Display coastlines
        coastlinecolor="black",  # Color of the coastline
        showcountries=True,  # Display country borders
        countrycolor="black",  # Color of the country borders
        projection_type="orthographic",  # Projection style (more globe-like)
        showframe=False,  # Turn off the frame around the map
    ),
    title="Disease Observations Worldwide",
    geo_center=dict(lon=0, lat=0),  # Center the map at (0,0) for better coverage
    geo_projection_scale=1,  # Adjust the scale to make the globe more zoomed in or out
)

# Show the plot
fig.show()


In [15]:
# Create the scatter_geo plot
fig = px.scatter_geo(diseased_tomatoes_df,
                     lon='longitude',
                     lat='latitude',
                     color='disease',  # Optionally color by disease or another column
                     hover_name='disease',  # Information to show when hovering over a point
                     hover_data={'actual_cc': True},  # Add 'actual_cc' to hover data
                     title="Global Disease Observations",
                    #  projection="orthographic",  # Use 'orthographic' for a globe-like appearance
                     color_continuous_scale=px.colors.sequential.Plasma)

# Show the plot
fig.show()


In [16]:
# Create the scatter_geo plot
fig = px.scatter_geo(diseased_tomatoes_df,
                     lon='longitude',
                     lat='latitude',
                     color='disease',  # Optionally color by disease or another column
                     hover_name='disease',  # Information to show when hovering over a point
                     hover_data={'actual_cc': True},  # Add 'actual_cc' to hover data
                     title="Global Disease Observations",
                     projection="orthographic",  # Use 'orthographic' for a globe-like appearance
                     color_continuous_scale=px.colors.sequential.Plasma)

# Customize layout for better display and to show climate-related features
fig.update_layout(
    geo=dict(
        showland=True,  # Show land on the map
        landcolor="lightgray",  # Land color
        showocean=True,  # Show ocean
        oceancolor="lightblue",  # Ocean color
        showcoastlines=True,  # Display coastlines
        coastlinecolor="black",  # Color of the coastline
        showcountries=False,  # Remove country borders
        showframe=False,  # Turn off the frame around the map
        showlakes=True,  # Optionally, show lakes
        lakecolor="lightblue",  # Color for lakes
        projection_type="orthographic",  # Projection style (globe-like)
        # Optional: If you have specific climatic zone data, you can add it here
    ),
    title="Disease Observations with Climatic Zones",
    geo_center=dict(lon=0, lat=0),  # Center the map at (0,0) for better coverage
    geo_projection_scale=1,  # Adjust the scale to make the globe more zoomed in or out
)

# Show the plot
fig.show()

In [17]:
# Function to get the full country name from ISO country code
def get_country_name(iso_code):
    try:
        country = pycountry.countries.get(alpha_2=iso_code)  # alpha_2 is the 2-letter ISO code
        return country.name if country else iso_code  # If country is found, return the full name
    except KeyError:
        return iso_code  # If not found, return the ISO code itself

# Map the country codes to their full names using pycountry
country_names = all_diseases_country_counts.index.map(get_country_name)

# Create a DataFrame for Plotly
all_diseases_counts_full_country_names = pd.DataFrame({
    'Country': country_names,
    'Count': all_diseases_country_counts.values
})

# Create a Plotly bar chart
fig = px.bar(all_diseases_counts_full_country_names, x='Country', y='Count',
             title="Country Distribution for All Diseases",
             labels={'Country': 'Country', 'Count': 'Count'},
             color='Country')

# Adjust layout for better display
fig.update_layout(
    xaxis_title="Country",
    yaxis_title="Count",
    xaxis_tickangle=-45,
    xaxis={'categoryorder': 'total descending'},  # Order by count
    legend_title="Countries"
)

fig.show()

In [18]:
# Function to get the continent for a given ISO country code
def iso_country_to_continent(iso_country):
    country_name = get_country_name(iso_country)
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

# Map the country codes to their continents using pycountry_convert
continents = all_diseases_country_counts.index.map(iso_country_to_continent)

# Create a DataFrame for Plotly
all_diseases_counts_by_continent = pd.DataFrame({
    'Continent': continents,
    'Count': all_diseases_country_counts.values
})

# Aggregate counts by continent
continent_counts = all_diseases_counts_by_continent.groupby('Continent')['Count'].sum().reset_index()

# Create a Plotly bar chart for continents
fig = px.bar(continent_counts, x='Continent', y='Count',
             title="Country Distribution by Continent for All Diseases",
             labels={'Continent': 'Continent', 'Count': 'Count'},
             color='Continent')

# Adjust layout for better display
fig.update_layout(
    xaxis_title="Continent",
    yaxis_title="Count",
    xaxis_tickangle=-45,
    xaxis={'categoryorder': 'total descending'},  # Order by count
    legend_title="Continents"
)

fig.show()


In [19]:
# Count occurrences of each country grouped by disease
disease_country_counts = diseased_tomatoes_df.groupby('disease')['actual_cc'].value_counts()

# Display the results
print("\nTop countries for each disease:")
for disease, group in disease_country_counts.groupby(level=0):
    print(f"\nDisease: {disease}")
    print(group)


Top countries for each disease:

Disease: Early Blight
disease       actual_cc
Early Blight  US           20
              CK            3
Name: count, dtype: int64

Disease: Late Blight
disease      actual_cc
Late Blight  BE           97
             US           29
             RU           22
             BY           20
             CZ           19
             DE           18
             AT           12
             PL           12
             TW           12
             CA           11
             PT           11
             FI            8
             GB            7
             AU            6
             ES            6
             UA            6
             CO            5
             FR            5
             IT            5
             CR            2
             LT            2
             NZ            2
             ZA            2
             NL            1
Name: count, dtype: int64

Disease: Leaf Miner
disease     actual_cc
Leaf Miner  US          

In [20]:
diseased_tomatoes_df.columns

Index(['observation_id', 'time_observed_at_date', 'observed_on_date',
       'observed_on_year', 'observed_on_month', 'observed_on_week',
       'observed_on_day', 'observed_on_hour', 'location', 'place_guess',
       'observed_time_zone', 'created_time_zone', 'time_zone_offset',
       'english_common_name', 'preferred_common_name', 'taxon_name',
       'description', 'identifications_most_disagree',
       'identifications_most_agree', 'quality_grade', 'uri', 'image_url',
       'original_image_url', 'original_width', 'original_height',
       'license_code', 'attribution', 'latitude', 'longitude',
       'actual_city_name', 'actual_admin1', 'actual_admin2', 'actual_cc',
       'roboflow_file_name', 'identifier', 'disease'],
      dtype='object')

In [21]:
import plotly.express as px

def plot_distribution_plotly(df, column_name):
    """
    Plots the distribution of values in a specified column of a DataFrame using Plotly.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column_name (str): The column for which to display the distribution.
    """
    if column_name not in df.columns:
        print(f"Column '{column_name}' does not exist in the DataFrame.")
        return

    # Count the occurrences of each value
    value_counts = df[column_name].value_counts().reset_index()
    value_counts.columns = [column_name, 'count']

    # Plot using Plotly
    fig = px.bar(value_counts,
                 x=column_name,
                 y='count',
                 title=f"Distribution of '{column_name}'",
                 labels={column_name: column_name, 'count': 'Count'},
                 text='count',
                 color=column_name)  # Optional coloring by category
    fig.update_traces(textposition='outside')
    fig.update_layout(
        xaxis_title=column_name,
        yaxis_title="Count",
        showlegend=False
    )
    fig.show()

In [22]:
plot_distribution_plotly(diseased_tomatoes_df, 'license_code')

In [23]:
plot_distribution_plotly(diseased_tomatoes_df, 'quality_grade')

In [24]:
plot_distribution_plotly(diseased_tomatoes_df[diseased_tomatoes_df['quality_grade'] == 'research'], 'license_code')

In [25]:
# Step 1: Encode the 'disease' column
def encode_disease_column(df, column_name):
    df = df.copy()  # Avoid modifying the original DataFrame
    le = LabelEncoder()
    df[f"{column_name}_encoded"] = le.fit_transform(df[column_name])
    return df

# Apply encoding
diseased_tomatoes_df = encode_disease_column(diseased_tomatoes_df, 'disease')

# Step 2: Select columns of interest
columns_of_interest = ['observed_on_year', 'observed_on_month', 'observed_on_week', 'disease_encoded']
correlation_df = diseased_tomatoes_df[columns_of_interest]

# Step 3: Compute correlation matrix and round to 2 decimal places
correlation_matrix = correlation_df.corr().round(2)

# Step 4: Visualize correlation matrix using Plotly
fig = px.imshow(correlation_matrix,
                text_auto=True,  # Annotate cells with correlation values
                color_continuous_scale='Viridis',
                title="Correlation Heatmap: Time Observations and Disease",
                labels=dict(color="Correlation"))
fig.update_layout(
    xaxis_title="Variables",
    yaxis_title="Variables",
    coloraxis_colorbar=dict(title="Correlation Coefficient")
)
fig.show()


In [26]:
# Step 1: Create a season column
def map_month_to_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

diseased_tomatoes_df['season'] = diseased_tomatoes_df['observed_on_month'].apply(map_month_to_season)

# Step 2: Create a contingency table
contingency_table = pd.crosstab(diseased_tomatoes_df['season'], diseased_tomatoes_df['disease'])

# Step 3: Perform the Chi-Square Test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Step 4: Output results
print("Chi-Square Statistic:", chi2)
print("P-value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies Table:\n", expected)

# Interpret results
if p < 0.05:
    print("There is a significant association between season and disease.")
else:
    print("No significant association between season and disease.")

Chi-Square Statistic: 164.9627710385117
P-value: 8.867814327731438e-26
Degrees of Freedom: 18
Expected Frequencies Table:
 [[9.72773973e+00 1.35342466e+02 3.25667808e+01 2.49537671e+01
  2.62226027e+01 1.73407534e+01 8.45890411e-01]
 [1.89041096e+00 2.63013699e+01 6.32876712e+00 4.84931507e+00
  5.09589041e+00 3.36986301e+00 1.64383562e-01]
 [1.01215753e+01 1.40821918e+02 3.38852740e+01 2.59640411e+01
  2.72842466e+01 1.80428082e+01 8.80136986e-01]
 [1.26027397e+00 1.75342466e+01 4.21917808e+00 3.23287671e+00
  3.39726027e+00 2.24657534e+00 1.09589041e-01]]
There is a significant association between season and disease.


In [27]:
fig = px.imshow(contingency_table,
                text_auto=True,
                color_continuous_scale="Viridis",
                title="Disease Occurrence by Season")
fig.update_layout(xaxis_title="Disease", yaxis_title="Season")
fig.show()

In [28]:
diseased_tomatoes_df['observed_on_month'].value_counts()

,count
observed_on_month,
10,177
7,118
8,91
9,56
6,48
4,20
5,16
1,15
11,14


In [45]:
diseased_tomatoes_df['identifications_most_agree'].count()

584

In [46]:
diseased_tomatoes_df['identifications_most_disagree'].count()

584

In [51]:
import plotly.express as px
import pandas as pd

# Example dataset (replace with your actual dataset)
# Assume 'diseased_tomatoes_df' has Boolean fields 'identifications_most_agree' and 'identifications_most_disagree'

# Step 1: Aggregate by disease, counting the number of True values
agreement_disagreement_df = diseased_tomatoes_df.groupby('disease').agg({
    'identifications_most_agree': 'sum',  # Count True values
    'identifications_most_disagree': 'sum'  # Count True values
}).reset_index()

# Step 2: Create a new column for the total number of identifications (agree + disagree)
agreement_disagreement_df['total_identifications'] = agreement_disagreement_df['identifications_most_agree'] + agreement_disagreement_df['identifications_most_disagree']

# Step 3: Plot 1 - Bar Plot of Agreement and Disagreement Counts for each Disease
fig1 = px.bar(agreement_disagreement_df,
              x='disease',
              y=['identifications_most_agree', 'identifications_most_disagree'],
              title='Agreement vs Disagreement Counts by Disease',
              labels={'value': 'Count', 'disease': 'Disease'},
              barmode='stack')
fig1.show()


In [52]:
early_blight_rows = diseased_tomatoes_df[diseased_tomatoes_df['disease'] == 'Early Blight']
print(early_blight_rows[['identifications_most_agree', 'identifications_most_disagree']].value_counts())

identifications_most_agree  identifications_most_disagree
False                       False                            23
Name: count, dtype: int64


In [47]:
# Check for missing values or zeros
early_blight_data = diseased_tomatoes_df[diseased_tomatoes_df['disease'] == 'Early Blight']
print(early_blight_data[['identifications_most_agree', 'identifications_most_disagree']].isnull().sum())
print(early_blight_data[['identifications_most_agree', 'identifications_most_disagree']].describe())


identifications_most_agree       0
identifications_most_disagree    0
dtype: int64
       identifications_most_agree identifications_most_disagree
count                          23                            23
unique                          1                             1
top                         False                         False
freq                           23                            23


In [37]:
def plot_class_distribution(class_counts, title="Class Distribution"):
    """
    Plots a distribution of classes using a dictionary of class counts.

    Args:
    - class_counts (dict): Dictionary with class names as keys and counts as values.
    - title (str): Title for the plot.

    Returns:
    - None: Displays the plot.
    """
    # Convert the dictionary to a DataFrame for Plotly
    data = {
        "Class": list(class_counts.keys()),
        "Count": list(class_counts.values())
    }

    # Create the bar plot
    fig = px.bar(data,
                 x="Class",
                 y="Count",
                 text="Count",
                 color="Class",
                 title=title,
                 labels={"Class": "Disease Class", "Count": "Number of Images"},
                 color_discrete_sequence=px.colors.qualitative.Plotly)

    # Enhance appearance
    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(
        xaxis=dict(title="Class", tickangle=45),
        yaxis=dict(title="Count"),
        showlegend=False,
        title=dict(x=0.5),  # Center the title
        height=700,
        width=700
    )

    # Show the plot
    fig.show()

In [38]:
raw_data_class_counts = {
    "Early Blight": 23,
    "Late Blight": 337,
    "Leaf Miner": 80,
    "Leaf Mold": 67,
    "Septoria": 62,
    "Spider Mites": 47,
    "Yellow Leaf Curl Virus": 2
}

plot_class_distribution(raw_data_class_counts, title="Disease Class Distribution(RawDataVersion)")

In [39]:
# Total images: 584
filtered_data_class_counts = {
	"Early Blight": 23,
	"Late Blight": 320,
	"Leaf Miner": 77,
	"Leaf Mold": 59,
	"Septoria": 62,
	"Spider Mites": 41,
	"Yellow Leaf Curl Virus": 2
}

plot_class_distribution(filtered_data_class_counts, title="Disease Class Distribution(FilteredDataVersion)")

In [60]:
import os
import cv2
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm

# Step 1: Define the base directory and load all image paths
base_dir = "/content/inaturalist-diseased-tomatoes-3"
data = []

for split in ['train', 'valid', 'test']:
    split_dir = os.path.join(base_dir, split)
    for root, _, files in os.walk(split_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Include image files only
                class_name = os.path.basename(root)  # Subdirectory name as class
                file_path = os.path.join(root, file)
                data.append({'image_path': file_path, 'class': class_name})

# Create a DataFrame for the dataset
dataset_df = pd.DataFrame(data)
print(f"Total images found: {len(dataset_df)}")
print("Sample data:")
print(dataset_df.head())

# Step 2: Define a function to calculate image entropy
def calculate_entropy(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])  # Calculate histogram
    hist_prob = hist / hist.sum()  # Normalize histogram to probabilities
    entropy = -np.sum(hist_prob * np.log2(hist_prob + 1e-10))  # Calculate entropy
    return entropy

# Step 3: Calculate entropy for all images
print("Calculating entropy for all images...")
dataset_df['entropy'] = dataset_df['image_path'].apply(calculate_entropy)

# Step 4: Plot histogram distribution of entropy values by class
fig = px.histogram(
    dataset_df,
    x='entropy',
    color='class',
    title="Histogram Distribution of Image Complexity(Entropy) by Class",
    labels={'entropy': 'Image Entropy', 'class': 'Class'},
    nbins=50,  # Adjust the number of bins for granularity
    barmode='overlay',  # Bars are overlaid
    opacity=0.75,  # Transparency for overlapping bars
    hover_data={'class': True, 'entropy': ':.2f'}  # Explicit hover info
)
fig.update_layout(
    xaxis_title="Image Entropy",
    yaxis_title="Count",
    legend_title="Class"
)
fig.show()


Total images found: 584
Sample data:
                                          image_path        class
0  /content/inaturalist-diseased-tomatoes-3/train...  Late Blight
1  /content/inaturalist-diseased-tomatoes-3/train...  Late Blight
2  /content/inaturalist-diseased-tomatoes-3/train...  Late Blight
3  /content/inaturalist-diseased-tomatoes-3/train...  Late Blight
4  /content/inaturalist-diseased-tomatoes-3/train...  Late Blight
Calculating entropy for all images...
